In [1]:
import pandas as pd
import numpy as np
#!pip install pandas --upgrade
df = pd.read_csv('crimes.csv')

In [2]:
poverty = pd.read_csv('poverty.csv')
poverty = poverty.drop('COMMUNITY AREA NAME', 1)
poverty.head()

,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,0.3,7.5,5.2,4.5,26.2,57123,6


In [3]:
#  Remove unwanted columns from the data
# Add 'Date' back in when RJ fixes the datetime converter
useful_columns = ['IUCR', 'Location Description', 'Arrest', 'Domestic', 'Community Area', 'Year']
data = df[useful_columns]

In [4]:
# Merge with the poverty dataframe
data.rename(columns={'Community Area': 'Community Area Number'}, inplace=True)
data = pd.merge(data, poverty, how='right', on='Community Area Number')

/Users/kelseyball/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [5]:
data.head()

,IUCR,Location Description,Arrest,Domestic,Community Area Number,Year,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,2820,RESIDENCE,False,True,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
1,0560,SIDEWALK,True,False,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
2,0460,STREET,False,False,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
3,1811,SIDEWALK,True,False,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
4,0460,STREET,False,False,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96


In [15]:
from sklearn.preprocessing import LabelEncoder
features = ['IUCR', 'Location Description', 'Arrest', 'Domestic']
for a in features:
    le = LabelEncoder()
    le.fit(data[a])
    data[a] = le.transform(data[a])
data.head()

,IUCR,Location Description,Arrest,Domestic,Community Area Number,Year,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,312,132,0,1,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
1,81,148,1,0,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
2,41,152,0,0,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
3,257,148,1,0,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96
4,41,152,0,0,30,2012,15.2,30.7,15.8,54.8,33.8,10402,96


In [18]:
from sklearn.cross_validation import train_test_split

data = data.dropna()
X = data.drop(['Arrest'], axis=1)
y = data['Arrest'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

logr = linear_model.LogisticRegression()
logr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
logr.score(X_test, y_test)

0.77308677638618439

In [23]:
# Todo: gridsearchCV for best params
# ROC curve

probs = logr.predict_proba(x_test)
print probs


[[ 0.8600153   0.1399847 ]
 [ 0.42198732  0.57801268]
 [ 0.76285514  0.23714486]
 ..., 
 [ 0.60499482  0.39500518]
 [ 0.77966898  0.22033102]
 [ 0.4804173   0.5195827 ]]
